<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parte 1 - Solicitação de dados primários ao usuário

In [ ]:
# Pedir o nome da cidade e o total de resíduos produzidos por kg/mês na cidade
cidade = input("Informe o nome da cidade: ")
residuos_cidade = int(input("Informe a quantidade, em kg, de resíduos gerados por mês na cidade: "))

# Pedir ao usuário a quantidade de bairros
n_bairros = int(input("Informe o número de bairros que a cidade possui: "))

bairros = [] # Lista para armazenar os nomes dos bairros
habitantes_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Pedir ao usuário os nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f"Digite o nome do {i+1}º bairro: ")
    bairros.append(bairro)

# Pedir ao usuário o número de habitantes de cada bairro e validar se é um número inteiro válido
for bairro in bairros:
    while True:
        try:
            habitantes = int(input(f"Digite o número de habitantes do bairro {bairro}: "))
            if habitantes < 0:
                print("Por favor, insira um número maior ou igual a zero para o número de habitantes.")
                continue
            else:
                habitantes_bairro.append(habitantes)
                break
        except ValueError:
            print("Por favor, insira um número inteiro para o número de habitantes.")

Informe o nome da cidade: Natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 300
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: Lagoa
Digite o nome do 2º bairro: Aqua 
Digite o número de habitantes do bairro Lagoa: 20
Digite o número de habitantes do bairro Aqua : 40


Parte 2 - Calculando a quantidade em kg de resíduos sólidos gerados em cada bairro por semana - considerando o valor fornecido da geração mensal da cidade.

In [ ]:
# Calculando a quantidade de resíduos sólidos gerados em cada bairro por semana
total_habitantes = sum(habitantes_bairro) #sum soma os valores da lista

residuos_por_bairro_semanal = []

for i, habitantes in enumerate(habitantes_bairro):
    proporcao = habitantes / total_habitantes
    residuos_bairro_semanal = round((proporcao * residuos_cidade) / 4)  # Dividindo por 4 para obter a quantidade semanal
    # round arredonda o valor para o inteiro mais próximo
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

Parte 3 - Informando ao usuário os resultados obtidos

In [ ]:
print(f"Cidade: {cidade}")
print("Bairros e seus respectivos números de habitantes:")
for bairro, habitantes in zip(bairros, habitantes_bairro):
  print(f"{bairro}: {habitantes} habitantes")


for bairro, residuos_semanais in zip(bairros, residuos_por_bairro_semanal):
  print(f"No bairro {bairro}, a quantidade de resíduos por semana é de aproximadamente {residuos_semanais} kg.")

Cidade: Natal
Bairros e seus respectivos números de habitantes:
Lagoa : 20 habitantes
Pneu: 40 habitantes
No bairro Lagoa , a quantidade de resíduos por semana é de aproximadamente 25 kg.
No bairro Pneu, a quantidade de resíduos por semana é de aproximadamente 50 kg.


Quantos caminhões serão necessarios

In [ ]:
#calculando quantos caminhões seriam necessario
caminhão = 7000
Quantidade_de_caminhões_necessarios = math.ceil(residuos_semanais/caminhão)
print(f"Quantidade de caminhões necessários: {Quantidade_de_caminhões_necessarios}")

Calculo dos gastos

In [ ]:
Coleta = float(1.35*residuos_cidade)
transporte = float(0.65*residuos_cidade)
Aterro = float(0.45*residuos_cidade)
print (f"Valor da coleta: {Coleta}")
print (f"Valor do transporte: {transporte}")
print (f"Valor do aterro: {Aterro}")
print (f"Valor total: {Coleta + transporte +Aterro}")

Valor da coleta: 405.0
Valor do transporte: 195.0
Valor do aterro: 135.0
Valor total: 735.0


Quais e quantas estações de coleta o bairro precisa ?

In [ ]:
import math

# Capacidade que uma empresa de pequeno porte pode processar (em kg)
capacidade_empresa_pequena = 500000

# Capacidade que uma empresa de médio porte pode processar (em kg)
capacidade_empresa_media = 2000000

# Calculando quantas empresas seriam necessárias para cada tipo de estação de coleta
empresas_necessarias_pequena = math.ceil(residuos_cidade / capacidade_empresa_pequena)
empresas_necessarias_media = math.ceil(residuos_cidade / capacidade_empresa_media)

# Exibindo os resultados
print("Número estimado de empresas necessárias para processar os resíduos sólidos:")
print(f"Para pequenas estações de coleta:")
print(f"- Necessário: {empresas_necessarias_pequena} empresas")

print(f"Para médias estações de coleta:")
print(f"- Necessário: {empresas_necessarias_media} empresas")



Número estimado de empresas necessárias para processar os resíduos sólidos:
Para pequenas estações de coleta:
- Necessário: 1 empresas
Para médias estações de coleta:
- Necessário: 1 empresas
